____
# Secretaría de Paz y Cultura Ciudadana - SPCC
## Oservatorio de Paz y Convivencia - OPC
### PROCEDIMIENTO PARA CURACION DE DATOS BD PONAL
#### Equipo OPC: 
##### Andrés F. Suárez, Mauricio Sedano  
##### Ana M. Gálvez, Karol C. Benítez
##### Juan M. Torres, Sergio Hernadez  
#### Actualización 17ABR2019
----

____

## CALCULO DE DENSIDAD y TASAS

En esta sección se hará el cálculo de los datos densidad ajustada y tasas de todas las DB de los delitos a nivel cali. 

Los datos a cargar debera  ser  dataframe merge "df_delito_2010a2019_raw_fecha.csv

Las operaciones a realizar son:

### ETAPA 1. DENSIDAD AJUSTADA
1. Cargar la DB curada que compila los 6 delitos ('AMENAZA','DELITO SEXUAL','HOMICIDIO','HURTO PERSONA', 'LESIONES PERSONALES', 'VIOLENCIA INTRAFAMILIAR')
2. Realizar reportes por conflictividad ["CONFLICTIVIDAD"]

3. MERGE todos las seis (6) DB para generar una (1) DB compilada. Para aplicar tratamiento como un todo. 
4. Verificacion de datos NULL y NaN (Sacar estadisticas)
5. Ajuste del formato de la columna FECHA y  derivar columnas ['FECHA_FORMATO'], ['AÑO'], ['MES'], ['DIA_NUMERO'], ['SEMANA_NUMERO'], ['DIA_JULIANO'], ['FECHA_ESPECIAL']
6. Ajuste del formato de la columna HORA y  derivar columnas ['HORA_FORMATO_24h'],['HORA_RANGO_24x']
7. Ajuste de la columna ['SEXO'] en la misma columna. Se mapea con el diccionario 
{'MASCULINO':'HOMBRE', 'FEMENINO':'MUJER', '-':'SIN DATO', 'NO REPORTADO':'SIN DATO', 'NO REPORTA':'SIN DATO'}
8. Ajuste del formato a númerico de la columna ['EDAD'] y derivar columnas ['EDAD_FORMATO'],['EDAD_5Q_80mas'],['EDAD_NNAJ']. Se usan los siguientes criterior para depurar la columna ['EDAD_FORMATO'], a) ['EDAD_FORMATO'] < 0 = np.nan b) ['EDAD_FORMATO'] > 115 = np.nan
9. Ajueste campo ['BARRIO'] y derivar columnas ['BARRIO_TEMP'], ['ESTACION_COD_DEV'], [COMUNA_DEV]

### ETAPA 2. TASAS

1. Seleccionar y crear dataframe para datos de sólo CALI (CT) con los 6 delitos ('AMENAZA','DELITO SEXUAL','HOMICIDIO','HURTO PERSONA', 'LESIONES PERSONALES', 'VIOLENCIA INTRAFAMILIAR')
2. Limpiar la columna ['BARRIO_NOMBRE'] aplicando diccionario
3. Mapeo columna ['BARRIO_OFICIAL'] con bases de datos oficial y derivar ['BARRIO_COD'], ['COMUNA'],['AREA'] 

____

### **ETAPA 1. FORMATOS y DERIVADOS**

#### **1.1 CARGAR ARCHIVO .CSV DEPURADO **

> **NOTA**
Se encuentra en la ruta ./df_2010u2017_pg_cali seleccionar el mas reciente

In [82]:
import pandas as pd

nombre_archivo ='./df_2010u2017_pg_cali/df_gen_2010a2019_cali_compact_20190421-2023.csv'
df_gen_2010a2018 = pd.read_csv(nombre_archivo, )

print('El tamaño del dataframe es:', df_gen_2010a2018.shape)


El tamaño del dataframe es: (217449, 38)


C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (23,28,30,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [83]:
df_gen_2010a2018.sample(5)

,FECHA_FORMATO,AÑO,MES,SEMANA_NUMERO,DIA_NUMERO,DIA,FECHAS_ESPECIALES,HORA_FORMATO_24h,HORA_24x,ZONA,...,COMUNA_DEV,CGTO_NOMBRE,VEREDA_NOMBRE,INVASION_NOMBRE,ZOI_NOMBRE,METROPOLITANA_NOMBRE,BARRIO_OFICIAL,COD_BARRIO,COMUNA_OFICIAL,AREA_km2
147892,2013-12-01,2013.0,12.0,48.0,1.0,DOMINGO,NO ES FECHA ESPECIAL,02:18:00,3,URBANA,...,07,SIN DATO,NaN,NaN,NaN,NaN,ALFONSO LOPEZ P. 3A. ETAPA,0703,07,0.579420
112158,2018-03-14,2018.0,3.0,11.0,14.0,MIÉRCOLES,NO ES FECHA ESPECIAL,09:30:00,10,URBANA,...,15,SIN DATO,NaN,NaN,NaN,NaN,EL MORICHAL,1599,15,1.196389
81762,2015-08-23,2015.0,8.0,34.0,23.0,DOMINGO,NO ES FECHA ESPECIAL,11:43:00,12,URBANA,...,02,SIN DATO,NaN,NaN,NaN,NaN,SAN VICENTE,0209,02,0.806922
120244,2018-09-17,2018.0,9.0,38.0,17.0,LUNES,NO ES FECHA ESPECIAL,15:00:00,16,URBANA,...,03,SIN DATO,NaN,NaN,NaN,NaN,SANTA ROSA,0307,03,0.189782
129426,2010-05-19,2010.0,5.0,20.0,19.0,MIÉRCOLES,NO ES FECHA ESPECIAL,09:30:00,10,URBANA,...,13,SIN DATO,NaN,NaN,NaN,NaN,EL VERGEL,1302,13,0.380952


#### ** RESPALDO ** 
____

In [84]:
#RESPALDO
df_gen_2010a2018_ = df_gen_2010a2018.copy()

#df_gen_2010a2018 = df_gen_2010a2018_.copy()

____

In [85]:
import numpy as np
df_gen_2010a2018 = df_gen_2010a2018[np.isfinite(df_gen_2010a2018['AÑO'])]

In [86]:
df_gen_2010a2018.AÑO = df_gen_2010a2018.AÑO.astype(int)
df_gen_2010a2018.AÑO = df_gen_2010a2018.AÑO.apply(str)
print(df_gen_2010a2018.AÑO.unique())

['2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017' '2018' '2019']


In [87]:
print(df_gen_2010a2018.COMUNA_OFICIAL.unique())

['14' '15' '07' '18' '06' '05' '20' '11' '04' '19' '03' 'SIN DATO' '02'
 '01' '10' '12' '09' '08' '13' '16' '17' '21' '22']


In [88]:
print(df_gen_2010a2018.COD_BARRIO.unique())
#df_gen_2010a2018.COD_BARRIO = df_gen_2010a2018.COD_BARRIO.astype(int, errors='ignore')
#df_gen_2010a2018.COD_BARRIO = df_gen_2010a2018.COD_BARRIO.apply(str).str.zfill(4)
#df_gen_2010a2018.COD_BARRIO = df_gen_2010a2018.COD_BARRIO
#print(df_gen_2010a2018.COD_BARRIO.unique())

['1401' '1504' '0707' '1802' '0610' '0504' '2002' '1117' '0410' '1906'
 '1904' '0597' '0311' 'SIN DATO' '0602' '0215' '0199' '1016' '0502' '0403'
 '1207' '1982' '1915' '1898' '1002' '0705' '1501' '1805' '1902' '2003'
 '1909' '0901' '0219' '1808' '0814' '1495' '0909' '1896' '0702' '1305'
 '0102' '0606' '1596' '0598' '0601' '0904' '1395' '1201' '0294' '0594'
 '0208' '0701' '0209' '1122' '1603' '1908' '1602' '1006' '1202' '1903'
 '1111' '1994' '1009' '0710' '1404' '0604' '1001' '0708' '1308' '0404'
 '1005' '1301' '1598' '0505' '1921' '0212' '1014' '1601' '1017' '1103'
 '0207' '1403' '1780' '1922' '0817' '0903' '1913' '0905' '0218' '1786'
 '2102' '0497' '0801' '0815' '0304' '1402' '2196' '0307' '2001' '0213'
 '1604' '1303' '0605' '0296' '0312' '0697' '1503' '1775' '1796' '1813'
 '0696' '1703' '1787' '1914' '1112' '1788' '2005' '1204' '0811' '2007'
 '1905' '0802' '0501' '0210' '1302' '1399' '0798' '2201' '1010' '0810'
 '1313' '1406' '1783' '1310' '1811' '1502' '2103' '2101' '0599' '1919'
 '

In [89]:
#USAR PARA VERIFICAR LAS COLUNAS EDAD SEA TIPO STRING Y SE VISUALICE TIPO 00-04 , 05-09, 10-14

print(df_gen_2010a2018.EDAD_NNAJ.unique())
print(df_gen_2010a2018.EDAD_5Q_80mas.unique())


['18-24' '25-29' '45-59' '30-44' '60+' '10-14' '15-17' '05-09' '00-04'
 'SIN DATO']
['20-24' '25-29' '55-59' '30-34' '40-44' '15-19' '50-54' '45-49' '35-39'
 '60-64' '10-14' '70-74' '65-69' '80+' '05-09' '75-79' '00-04' 'SIN DATO']


**Iniciamos con la sellección de solo datos de las 22 Comunas de Cali**

In [90]:
mask_cali = ((df_gen_2010a2018.COMUNA_OFICIAL !='23') & 
             (df_gen_2010a2018.COMUNA_OFICIAL !='24') & 
             (df_gen_2010a2018.COMUNA_OFICIAL !='25') &
             (df_gen_2010a2018.COMUNA_OFICIAL !='26') &
             (df_gen_2010a2018.COMUNA_OFICIAL !='27'))
df_gen_2010a2018_cali = df_gen_2010a2018[mask_cali]

In [91]:
print('cali urbano: ',len(df_gen_2010a2018_cali))
print('cali area metropolitana: ', len(df_gen_2010a2018))

cali urbano:  217014
cali area metropolitana:  217014


### **1.2 REPORTE POR CONFLICTIVIDAD **
> **Nota:**
Se realiza
- Eventos a nivel general

In [92]:
from IPython.display import display
display(df_gen_2010a2018_cali.sample(3))
#df_gen_2010a2018_cali.CANTIDAD.value_counts()

,FECHA_FORMATO,AÑO,MES,SEMANA_NUMERO,DIA_NUMERO,DIA,FECHAS_ESPECIALES,HORA_FORMATO_24h,HORA_24x,ZONA,...,COMUNA_DEV,CGTO_NOMBRE,VEREDA_NOMBRE,INVASION_NOMBRE,ZOI_NOMBRE,METROPOLITANA_NOMBRE,BARRIO_OFICIAL,COD_BARRIO,COMUNA_OFICIAL,AREA_km2
89169,2016-06-02,2016,6.0,22.0,2.0,JUEVES,NO ES FECHA ESPECIAL,03:51:00,4,URBANA,...,04,SIN DATO,NaN,NaN,NaN,NaN,CALIMA,0421,04,0.441970
216490,2018-12-02,2018,12.0,48.0,2.0,DOMINGO,NO ES FECHA ESPECIAL,22:30:00,23,URBANA,...,15,SIN DATO,NaN,NaN,NaN,NaN,LAUREANO GOMEZ,1503,15,0.223410
6157,2015-09-02,2015,9.0,36.0,2.0,MIÉRCOLES,NO ES FECHA ESPECIAL,15:00:00,16,URBANA,...,18,SIN DATO,NaN,NaN,NaN,NaN,BUENOS AIRES,1801,18,0.209696


In [93]:
columnas = ['SEXO', 'EDAD_NNAJ', 'CONFLICTIVIDAD', 'AÑO', 
            'COMUNA_OFICIAL', 'COD_BARRIO', 'BARRIO_OFICIAL',
            'AREA_km2', 'BARRIO_NOMBRE', 'CANTIDAD']

In [94]:
df_gen_resumen_conflictividad = pd.DataFrame() 
df_gen_resumen_conflictividad = df_gen_2010a2018_cali[columnas]
df_gen_resumen_conflictividad.sample(2)

,SEXO,EDAD_NNAJ,CONFLICTIVIDAD,AÑO,COMUNA_OFICIAL,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,BARRIO_NOMBRE,CANTIDAD
146900,HOMBRE,45-59,LESIONES PERSONALES,2013,21,2198,VALLE GRANDE,0.625452,VALLE GRANDE,1.0
215069,MUJER,30-44,VIOLENCIA INTRAFAMILIAR,2018,01,0101,TERRON COLORADO,1.386998,TERRON COLORADO,1.0


In [95]:
tabla_eventos_conflictividad_x_ano = pd.crosstab(index=df_gen_resumen_conflictividad.CONFLICTIVIDAD,
                                                  columns=df_gen_resumen_conflictividad.AÑO,
                                                  values=df_gen_resumen_conflictividad.CONFLICTIVIDAD,
                                                  aggfunc=len,
                                                  margins=True,
                                                  margins_name='Total'
                                                  )

tabla_eventos_conflictividad_x_ano


AÑO,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total
CONFLICTIVIDAD,,,,,,,,,,,
AMENAZA,805,948,919,585,1284,2212,1800,1692,2937,169,13351
DELITO SEXUAL,562,668,629,854,801,980,1293,1417,1420,210,8834
HOMICIDIO,1601,1814,1846,1960,1544,1348,1281,1239,1149,76,13858
HURTO PERSONA,6518,7429,7496,9288,9130,8647,10357,13786,15731,2671,91053
LESIONES PERSONALES,4766,5067,4916,6102,5416,5701,6664,7230,7974,1230,55066
VIOLENCIA INTRAFAMILIAR,2332,3558,3880,3010,3269,4247,4616,4630,4613,697,34852
Total,16584,19484,19686,21799,21444,23135,26011,29994,33824,5053,217014


**3.1 HOMICIDIOS** 

In [96]:
mask_ho = ((df_gen_resumen_conflictividad['CONFLICTIVIDAD']=='HOMICIDIO'))
df_gen_resumen_conflictividad_ho = pd.DataFrame() 
df_gen_resumen_conflictividad_ho = df_gen_resumen_conflictividad[mask_ho]

**3.1.3 Homicidios territorio barrio**

In [97]:
tabla_eventos_conflictividad_ho_x_barrio = pd.crosstab(index=[df_gen_resumen_conflictividad_ho.COD_BARRIO, 
                                                              df_gen_resumen_conflictividad_ho.BARRIO_OFICIAL, 
                                                              df_gen_resumen_conflictividad_ho.AREA_km2],
                                                       columns=df_gen_resumen_conflictividad_ho.AÑO,
                                                       values=df_gen_resumen_conflictividad_ho.CONFLICTIVIDAD,
                                                       aggfunc=len,
                                                       margins=False,
                                                       margins_name='Total'
                                                      )
tabla_eventos_conflictividad_ho_x_barrio = tabla_eventos_conflictividad_ho_x_barrio.fillna(0).astype(int)
tabla_eventos_conflictividad_ho_x_barrio


,,AÑO,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
COD_BARRIO,BARRIO_OFICIAL,AREA_km2,,,,,,,,,,
0101,TERRON COLORADO,1.386998,37,36,35,32,30,17,19,7,2,0
0102,VISTA HERMOSA,1.148898,0,0,0,5,0,6,3,3,5,0
0196,SECTOR PATIO BONITO,0.198371,0,0,0,0,0,0,0,1,1,0
0199,AGUACATAL,1.107991,0,9,3,17,5,2,3,3,1,0
0201,SANTA RITA,0.181149,1,2,1,0,2,0,0,0,0,0
0202,SANTA TERESITA,0.275827,1,1,1,0,1,1,3,0,1,0
0203,ARBOLEDAS,0.395990,0,0,0,0,0,0,1,0,0,0
0204,NORMANDIA,0.250467,0,0,3,0,1,1,0,1,2,0
0206,CENTENARIO,0.213702,1,0,1,1,0,1,1,0,0,0


In [98]:
data_frame_ref = pd.DataFrame(tabla_eventos_conflictividad_x_ano.to_records())

In [99]:
data_frame_ref.head()

,CONFLICTIVIDAD,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total
0,AMENAZA,805,948,919,585,1284,2212,1800,1692,2937,169,13351
1,DELITO SEXUAL,562,668,629,854,801,980,1293,1417,1420,210,8834
2,HOMICIDIO,1601,1814,1846,1960,1544,1348,1281,1239,1149,76,13858
3,HURTO PERSONA,6518,7429,7496,9288,9130,8647,10357,13786,15731,2671,91053
4,LESIONES PERSONALES,4766,5067,4916,6102,5416,5701,6664,7230,7974,1230,55066


In [100]:
data_frame_ref.iloc[0]['2010']

805

In [101]:
area_cali_km2=120.82

data_frame_ref['densidad_ref_2010']=data_frame_ref['2010']/area_cali_km2
data_frame_ref['densidad_ref_2011']=data_frame_ref['2011']/area_cali_km2
data_frame_ref['densidad_ref_2012']=data_frame_ref['2012']/area_cali_km2
data_frame_ref['densidad_ref_2013']=data_frame_ref['2013']/area_cali_km2
data_frame_ref['densidad_ref_2014']=data_frame_ref['2014']/area_cali_km2
data_frame_ref['densidad_ref_2015']=data_frame_ref['2015']/area_cali_km2
data_frame_ref['densidad_ref_2016']=data_frame_ref['2016']/area_cali_km2
data_frame_ref['densidad_ref_2017']=data_frame_ref['2017']/area_cali_km2
data_frame_ref['densidad_ref_2018']=data_frame_ref['2018']/area_cali_km2
data_frame_ref['densidad_ref_2019']=data_frame_ref['2019']/area_cali_km2

In [102]:
data_frame_ref

,CONFLICTIVIDAD,2010,2011,2012,2013,2014,2015,2016,2017,2018,...,densidad_ref_2010,densidad_ref_2011,densidad_ref_2012,densidad_ref_2013,densidad_ref_2014,densidad_ref_2015,densidad_ref_2016,densidad_ref_2017,densidad_ref_2018,densidad_ref_2019
0,AMENAZA,805,948,919,585,1284,2212,1800,1692,2937,...,6.662804,7.846383,7.606357,4.841914,10.627380,18.308227,14.898196,14.004304,24.308889,1.398775
1,DELITO SEXUAL,562,668,629,854,801,980,1293,1417,1420,...,4.651548,5.528886,5.206092,7.068366,6.629697,8.111240,10.701871,11.728191,11.753021,1.738123
2,HOMICIDIO,1601,1814,1846,1960,1544,1348,1281,1239,1149,...,13.251117,15.014071,15.278927,16.222480,12.779341,11.157093,10.602549,10.254925,9.510015,0.629035
3,HURTO PERSONA,6518,7429,7496,9288,9130,8647,10357,13786,15731,...,53.948022,61.488164,62.042708,76.874690,75.566959,71.569277,85.722562,114.103625,130.201953,22.107267
4,LESIONES PERSONALES,4766,5067,4916,6102,5416,5701,6664,7230,7974,...,39.447111,41.938421,40.688628,50.504883,44.827015,47.185896,55.156431,59.841086,65.999007,10.180434
5,VIOLENCIA INTRAFAMILIAR,2332,3558,3880,3010,3269,4247,4616,4630,4613,...,19.301440,29.448767,32.113888,24.913094,27.056779,35.151465,38.205595,38.321470,38.180765,5.768912
6,Total,16584,19484,19686,21799,21444,23135,26011,29994,33824,...,137.262043,161.264691,162.936600,180.425426,177.487171,191.483198,215.287204,248.253600,279.953650,41.822546


In [103]:

data_frame = pd.DataFrame(tabla_eventos_conflictividad_ho_x_barrio.to_records())
data_frame.head()


,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0101,TERRON COLORADO,1.386998,37,36,35,32,30,17,19,7,2,0
1,0102,VISTA HERMOSA,1.148898,0,0,0,5,0,6,3,3,5,0
2,0196,SECTOR PATIO BONITO,0.198371,0,0,0,0,0,0,0,1,1,0
3,0199,AGUACATAL,1.107991,0,9,3,17,5,2,3,3,1,0
4,0201,SANTA RITA,0.181149,1,2,1,0,2,0,0,0,0,0


In [104]:
data_frame.dtypes

COD_BARRIO         object
BARRIO_OFICIAL     object
AREA_km2          float64
2010                int32
2011                int32
2012                int32
2013                int32
2014                int32
2015                int32
2016                int32
2017                int32
2018                int32
2019                int32
dtype: object

In [105]:
data_frame.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0101,TERRON COLORADO,1.386998,37,36,35,32,30,17,19,7,2,0
1,0102,VISTA HERMOSA,1.148898,0,0,0,5,0,6,3,3,5,0
2,0196,SECTOR PATIO BONITO,0.198371,0,0,0,0,0,0,0,1,1,0
3,0199,AGUACATAL,1.107991,0,9,3,17,5,2,3,3,1,0
4,0201,SANTA RITA,0.181149,1,2,1,0,2,0,0,0,0,0


In [106]:
data_frame['densidad_esp_2010']=data_frame['2010']/data_frame['AREA_km2']
data_frame['densidad_esp_2011']=data_frame['2011']/data_frame['AREA_km2']
data_frame['densidad_esp_2012']=data_frame['2012']/data_frame['AREA_km2']
data_frame['densidad_esp_2013']=data_frame['2013']/data_frame['AREA_km2']
data_frame['densidad_esp_2014']=data_frame['2014']/data_frame['AREA_km2']
data_frame['densidad_esp_2015']=data_frame['2015']/data_frame['AREA_km2']
data_frame['densidad_esp_2016']=data_frame['2016']/data_frame['AREA_km2']
data_frame['densidad_esp_2017']=data_frame['2017']/data_frame['AREA_km2']
data_frame['densidad_esp_2018']=data_frame['2018']/data_frame['AREA_km2']
data_frame['densidad_esp_2019']=data_frame['2019']/data_frame['AREA_km2']


In [107]:
data_frame.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,...,densidad_esp_2010,densidad_esp_2011,densidad_esp_2012,densidad_esp_2013,densidad_esp_2014,densidad_esp_2015,densidad_esp_2016,densidad_esp_2017,densidad_esp_2018,densidad_esp_2019
0,0101,TERRON COLORADO,1.386998,37,36,35,32,30,17,19,...,26.676311,25.955330,25.234348,23.071404,21.629441,12.256683,13.698646,5.046870,1.441963,0.0
1,0102,VISTA HERMOSA,1.148898,0,0,0,5,0,6,3,...,0.000000,0.000000,0.000000,4.351995,0.000000,5.222394,2.611197,2.611197,4.351995,0.0
2,0196,SECTOR PATIO BONITO,0.198371,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.041068,5.041068,0.0
3,0199,AGUACATAL,1.107991,0,9,3,17,5,2,3,...,0.000000,8.122812,2.707604,15.343089,4.512673,1.805069,2.707604,2.707604,0.902535,0.0
4,0201,SANTA RITA,0.181149,1,2,1,0,2,0,0,...,5.520329,11.040658,5.520329,0.000000,11.040658,0.000000,0.000000,0.000000,0.000000,0.0


In [108]:
data_frame['eve_esperados_2010']=(data_frame_ref.iloc[0]['densidad_ref_2010'])*data_frame['AREA_km2']
data_frame['eve_esperados_2011']=(data_frame_ref.iloc[0]['densidad_ref_2011'])*data_frame['AREA_km2']
data_frame['eve_esperados_2012']=(data_frame_ref.iloc[0]['densidad_ref_2012'])*data_frame['AREA_km2']
data_frame['eve_esperados_2013']=(data_frame_ref.iloc[0]['densidad_ref_2013'])*data_frame['AREA_km2']
data_frame['eve_esperados_2014']=(data_frame_ref.iloc[0]['densidad_ref_2014'])*data_frame['AREA_km2']
data_frame['eve_esperados_2015']=(data_frame_ref.iloc[0]['densidad_ref_2015'])*data_frame['AREA_km2']
data_frame['eve_esperados_2016']=(data_frame_ref.iloc[0]['densidad_ref_2016'])*data_frame['AREA_km2']
data_frame['eve_esperados_2017']=(data_frame_ref.iloc[0]['densidad_ref_2017'])*data_frame['AREA_km2']
data_frame['eve_esperados_2018']=(data_frame_ref.iloc[0]['densidad_ref_2018'])*data_frame['AREA_km2']
data_frame['eve_esperados_2019']=(data_frame_ref.iloc[0]['densidad_ref_2019'])*data_frame['AREA_km2']


In [109]:
data_frame.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,...,eve_esperados_2010,eve_esperados_2011,eve_esperados_2012,eve_esperados_2013,eve_esperados_2014,eve_esperados_2015,eve_esperados_2016,eve_esperados_2017,eve_esperados_2018,eve_esperados_2019
0,0101,TERRON COLORADO,1.386998,37,36,35,32,30,17,19,...,9.241299,10.882921,10.550004,6.715726,14.740158,25.393481,20.663773,19.423947,33.716390,1.940099
1,0102,VISTA HERMOSA,1.148898,0,0,0,5,0,6,3,...,7.654884,9.014696,8.738930,5.562866,12.209778,21.034291,17.116511,16.089521,27.928441,1.607050
2,0196,SECTOR PATIO BONITO,0.198371,0,0,0,0,0,0,0,...,1.321705,1.556492,1.508878,0.960494,2.108160,3.631815,2.955365,2.778043,4.822171,0.277476
3,0199,AGUACATAL,1.107991,0,9,3,17,5,2,3,...,7.382325,8.693720,8.427772,5.364795,11.775038,20.285346,16.507063,15.516639,26.934024,1.549830
4,0201,SANTA RITA,0.181149,1,2,1,0,2,0,0,...,1.206958,1.421361,1.377881,0.877106,1.925135,3.316510,2.698788,2.536860,4.403522,0.253386


In [110]:
data_frame['rme_2010']=data_frame['2010']/data_frame['eve_esperados_2010']
data_frame['rme_2011']=data_frame['2011']/data_frame['eve_esperados_2011']
data_frame['rme_2012']=data_frame['2012']/data_frame['eve_esperados_2012']
data_frame['rme_2013']=data_frame['2013']/data_frame['eve_esperados_2013']
data_frame['rme_2014']=data_frame['2014']/data_frame['eve_esperados_2014']
data_frame['rme_2015']=data_frame['2015']/data_frame['eve_esperados_2015']
data_frame['rme_2016']=data_frame['2016']/data_frame['eve_esperados_2016']
data_frame['rme_2017']=data_frame['2017']/data_frame['eve_esperados_2017']
data_frame['rme_2018']=data_frame['2018']/data_frame['eve_esperados_2018']
data_frame['rme_2019']=data_frame['2019']/data_frame['eve_esperados_2019']

In [111]:
data_frame.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,...,rme_2010,rme_2011,rme_2012,rme_2013,rme_2014,rme_2015,rme_2016,rme_2017,rme_2018,rme_2019
0,0101,TERRON COLORADO,1.386998,37,36,35,32,30,17,19,...,4.003766,3.307936,3.317534,4.764935,2.035256,0.669463,0.919484,0.360380,0.059318,0.0
1,0102,VISTA HERMOSA,1.148898,0,0,0,5,0,6,3,...,0.000000,0.000000,0.000000,0.898817,0.000000,0.285249,0.175269,0.186457,0.179029,0.0
2,0196,SECTOR PATIO BONITO,0.198371,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.359966,0.207375,0.0
3,0199,AGUACATAL,1.107991,0,9,3,17,5,2,3,...,0.000000,1.035230,0.355966,3.168807,0.424627,0.098593,0.181740,0.193341,0.037128,0.0
4,0201,SANTA RITA,0.181149,1,2,1,0,2,0,0,...,0.828529,1.407102,0.725752,0.000000,1.038888,0.000000,0.000000,0.000000,0.000000,0.0


In [112]:
columnas_ = ['COD_BARRIO',
             'BARRIO_OFICIAL', 
             '2010', 
             '2011', 
             '2012', 
             '2013', 
             '2014', 
             '2015', 
             '2016', 
             '2017', 
             '2018', 
             '2019',
             'rme_2010',
             'rme_2011',
             'rme_2012',
             'rme_2013',
             'rme_2014',
             'rme_2015',
             'rme_2016',
             'rme_2017',
             'rme_2018',
             'rme_2019']

In [113]:
data_frame_reporte_ho = data_frame[['COD_BARRIO', 
                                    'BARRIO_OFICIAL', 
                                    '2010',
                                    '2011',
                                    '2012',
                                    '2013',
                                    '2014',
                                    '2015',
                                    '2016',
                                    '2017',
                                    '2018',
                                    '2019',
                                    'rme_2010',
                                    'rme_2011',
                                    'rme_2012',
                                    'rme_2013',
                                    'rme_2014',
                                    'rme_2015',
                                    'rme_2016',
                                    'rme_2017',
                                    'rme_2018',
                                    'rme_2019']]
                                    
                                    
data_frame_reporte_ho.sample(10)



,COD_BARRIO,BARRIO_OFICIAL,2010,2011,2012,2013,2014,2015,2016,2017,...,rme_2010,rme_2011,rme_2012,rme_2013,rme_2014,rme_2015,rme_2016,rme_2017,rme_2018,rme_2019
128,1002,EL GUABAL,6,4,0,7,7,3,4,6,...,1.524943,0.863276,0.000000,2.448163,1.115401,0.277482,0.454659,0.725520,0.139323,0.000000
289,1983,TEJARES - CRISTALES,2,1,2,0,2,0,0,1,...,0.529865,0.224969,0.464136,0.000000,0.332197,0.000000,0.000000,0.126046,0.000000,0.000000
105,0807,BENJAMIN HERRERA,3,1,0,5,4,3,0,2,...,2.523104,0.714170,0.000000,5.786605,2.109136,0.918218,0.000000,0.800275,0.461037,0.000000
267,1903,EL LIDO,6,10,3,0,3,1,1,4,...,1.583384,2.240900,0.693484,0.000000,0.496349,0.096039,0.118021,0.502216,0.216994,0.000000
256,1813,ALFEREZ REAL,1,0,1,0,0,0,0,1,...,0.664372,0.000000,0.581958,0.000000,0.000000,0.000000,0.000000,0.316087,0.000000,0.000000
234,1784,LA HACIENDA,0,4,2,1,0,0,0,3,...,0.000000,0.706404,0.364348,0.286184,0.000000,0.000000,0.000000,0.296840,0.000000,0.000000
156,1112,PRADOS DE ORIENTE,4,1,3,3,2,0,0,1,...,2.121871,0.450450,1.393993,2.189879,0.665150,0.000000,0.000000,0.252380,0.145395,0.000000
65,0504,CHIMINANGOS SEGUNDA ETAPA,0,2,4,2,0,0,3,0,...,0.000000,1.248362,2.575512,2.022987,0.000000,0.000000,0.986206,0.000000,0.000000,0.000000
179,1302,EL VERGEL,30,35,34,30,33,10,20,16,...,11.819354,11.709223,11.733613,16.264239,8.151120,1.433782,3.523918,2.999080,3.023587,0.000000
207,1498,PROMOCIONES POPULARES B,0,5,3,1,3,3,0,5,...,0.000000,1.068923,0.661592,0.346441,0.473523,0.274866,0.000000,0.598900,1.380100,1.199217


In [114]:
import datetime
now = datetime.datetime.now()

ruta = './reportes/'
nombre_base = 'OPC_densidad_ajustada_ho'
fecha = str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'-'+str(now.hour).zfill(2)+str(now.minute).zfill(2)
nombre_archivo = ruta + nombre_base + fecha +'.xlsx'

writer = pd.ExcelWriter(nombre_archivo)
data_frame_reporte_ho.to_excel(writer, 'densidad')
writer.save()

**3.2 HURTO PERSONAS** 

In [115]:
mask_hp = ((df_gen_resumen_conflictividad['CONFLICTIVIDAD']=='HURTO PERSONA'))

df_gen_resumen_conflictividad_hp = pd.DataFrame() 
df_gen_resumen_conflictividad_hp = df_gen_resumen_conflictividad[mask_hp]

**3.2.3 Hurto personas territorio barrio **

In [116]:
tabla_eventos_conflictividad_hp_x_barrio = pd.crosstab(index=[df_gen_resumen_conflictividad_hp.COD_BARRIO, 
                                                              df_gen_resumen_conflictividad_hp.BARRIO_OFICIAL, 
                                                              df_gen_resumen_conflictividad_hp.AREA_km2],
                                                  columns=df_gen_resumen_conflictividad_hp.AÑO,
                                                  values=df_gen_resumen_conflictividad_hp.CONFLICTIVIDAD,
                                                  aggfunc=len,
                                                  margins=False,
                                                  margins_name='Total'
                                                  )
tabla_eventos_conflictividad_hp_x_barrio = tabla_eventos_conflictividad_hp_x_barrio.fillna(0).astype(int)
tabla_eventos_conflictividad_hp_x_barrio


,,AÑO,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
COD_BARRIO,BARRIO_OFICIAL,AREA_km2,,,,,,,,,,
0101,TERRON COLORADO,1.386998,51,67,55,69,67,100,107,105,76,18
0102,VISTA HERMOSA,1.148898,5,6,6,8,7,14,10,6,25,12
0196,SECTOR PATIO BONITO,0.198371,0,0,4,5,7,5,7,5,7,1
0199,AGUACATAL,1.107991,10,11,17,19,11,20,32,29,34,4
0201,SANTA RITA,0.181149,11,10,11,10,17,21,19,7,14,3
0202,SANTA TERESITA,0.275827,17,16,19,21,26,10,25,24,22,4
0203,ARBOLEDAS,0.395990,2,3,5,1,9,8,5,8,4,1
0204,NORMANDIA,0.250467,6,11,7,15,12,16,15,10,23,4
0205,JUANAMBU,0.221154,6,7,6,13,18,9,19,13,39,8


In [117]:
data_frame_hp = pd.DataFrame(tabla_eventos_conflictividad_hp_x_barrio.to_records())
data_frame_hp.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0101,TERRON COLORADO,1.386998,51,67,55,69,67,100,107,105,76,18
1,0102,VISTA HERMOSA,1.148898,5,6,6,8,7,14,10,6,25,12
2,0196,SECTOR PATIO BONITO,0.198371,0,0,4,5,7,5,7,5,7,1
3,0199,AGUACATAL,1.107991,10,11,17,19,11,20,32,29,34,4
4,0201,SANTA RITA,0.181149,11,10,11,10,17,21,19,7,14,3


In [118]:
data_frame_hp['densidad_esp_2010']=data_frame_hp['2010']/data_frame_hp['AREA_km2']
data_frame_hp['densidad_esp_2011']=data_frame_hp['2011']/data_frame_hp['AREA_km2']
data_frame_hp['densidad_esp_2012']=data_frame_hp['2012']/data_frame_hp['AREA_km2']
data_frame_hp['densidad_esp_2013']=data_frame_hp['2013']/data_frame_hp['AREA_km2']
data_frame_hp['densidad_esp_2014']=data_frame_hp['2014']/data_frame_hp['AREA_km2']
data_frame_hp['densidad_esp_2015']=data_frame_hp['2015']/data_frame_hp['AREA_km2']
data_frame_hp['densidad_esp_2016']=data_frame_hp['2016']/data_frame_hp['AREA_km2']
data_frame_hp['densidad_esp_2017']=data_frame_hp['2017']/data_frame_hp['AREA_km2']
data_frame_hp['densidad_esp_2018']=data_frame_hp['2018']/data_frame_hp['AREA_km2']
data_frame_hp['densidad_esp_2019']=data_frame_hp['2019']/data_frame_hp['AREA_km2']

In [119]:
data_frame_hp.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,...,densidad_esp_2010,densidad_esp_2011,densidad_esp_2012,densidad_esp_2013,densidad_esp_2014,densidad_esp_2015,densidad_esp_2016,densidad_esp_2017,densidad_esp_2018,densidad_esp_2019
0,0101,TERRON COLORADO,1.386998,51,67,55,69,67,100,107,...,36.770050,48.305752,39.653976,49.747715,48.305752,72.098138,77.145007,75.703044,54.794585,12.977665
1,0102,VISTA HERMOSA,1.148898,5,6,6,8,7,14,10,...,4.351995,5.222394,5.222394,6.963193,6.092794,12.185587,8.703991,5.222394,21.759977,10.444789
2,0196,SECTOR PATIO BONITO,0.198371,0,0,4,5,7,5,7,...,0.000000,0.000000,20.164271,25.205339,35.287474,25.205339,35.287474,25.205339,35.287474,5.041068
3,0199,AGUACATAL,1.107991,10,11,17,19,11,20,32,...,9.025346,9.927881,15.343089,17.148158,9.927881,18.050693,28.881108,26.173504,30.686178,3.610139
4,0201,SANTA RITA,0.181149,11,10,11,10,17,21,19,...,60.723620,55.203291,60.723620,55.203291,93.845595,115.926911,104.886253,38.642304,77.284607,16.560987


In [120]:
data_frame_hp['eve_esperados_2010']=(data_frame_ref.iloc[1]['densidad_ref_2010'])*data_frame_hp['AREA_km2']
data_frame_hp['eve_esperados_2011']=(data_frame_ref.iloc[1]['densidad_ref_2011'])*data_frame_hp['AREA_km2']
data_frame_hp['eve_esperados_2012']=(data_frame_ref.iloc[1]['densidad_ref_2012'])*data_frame_hp['AREA_km2']
data_frame_hp['eve_esperados_2013']=(data_frame_ref.iloc[1]['densidad_ref_2013'])*data_frame_hp['AREA_km2']
data_frame_hp['eve_esperados_2014']=(data_frame_ref.iloc[1]['densidad_ref_2014'])*data_frame_hp['AREA_km2']
data_frame_hp['eve_esperados_2015']=(data_frame_ref.iloc[1]['densidad_ref_2015'])*data_frame_hp['AREA_km2']
data_frame_hp['eve_esperados_2016']=(data_frame_ref.iloc[1]['densidad_ref_2016'])*data_frame_hp['AREA_km2']
data_frame_hp['eve_esperados_2017']=(data_frame_ref.iloc[1]['densidad_ref_2017'])*data_frame_hp['AREA_km2']
data_frame_hp['eve_esperados_2018']=(data_frame_ref.iloc[1]['densidad_ref_2018'])*data_frame_hp['AREA_km2']
data_frame_hp['eve_esperados_2019']=(data_frame_ref.iloc[1]['densidad_ref_2019'])*data_frame_hp['AREA_km2']

In [121]:
data_frame_hp.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,...,eve_esperados_2010,eve_esperados_2011,eve_esperados_2012,eve_esperados_2013,eve_esperados_2014,eve_esperados_2015,eve_esperados_2016,eve_esperados_2017,eve_esperados_2018,eve_esperados_2019
0,0101,TERRON COLORADO,1.386998,51,67,55,69,67,100,107,...,6.451689,7.668556,7.220841,9.803812,9.195379,11.250277,14.843477,16.266981,16.301421,2.410774
1,0102,VISTA HERMOSA,1.148898,5,6,6,8,7,14,10,...,5.344155,6.352128,5.981270,8.120834,7.616848,9.318989,12.295361,13.474498,13.503026,1.996926
2,0196,SECTOR PATIO BONITO,0.198371,0,0,4,5,7,5,7,...,0.922731,1.096769,1.032736,1.402157,1.315137,1.609032,2.122937,2.326529,2.331455,0.344793
3,0199,AGUACATAL,1.107991,10,11,17,19,11,20,32,...,5.153872,6.125954,5.768301,7.831684,7.345643,8.987178,11.857573,12.994726,13.022238,1.925824
4,0201,SANTA RITA,0.181149,11,10,11,10,17,21,19,...,0.842621,1.001550,0.943076,1.280425,1.200960,1.469340,1.938629,2.124546,2.129044,0.314859


In [122]:
data_frame_hp['rme_2010']=data_frame_hp['2010']/data_frame_hp['eve_esperados_2010']
data_frame_hp['rme_2011']=data_frame_hp['2011']/data_frame_hp['eve_esperados_2011']
data_frame_hp['rme_2012']=data_frame_hp['2012']/data_frame_hp['eve_esperados_2012']
data_frame_hp['rme_2013']=data_frame_hp['2013']/data_frame_hp['eve_esperados_2013']
data_frame_hp['rme_2014']=data_frame_hp['2014']/data_frame_hp['eve_esperados_2014']
data_frame_hp['rme_2015']=data_frame_hp['2015']/data_frame_hp['eve_esperados_2015']
data_frame_hp['rme_2016']=data_frame_hp['2016']/data_frame_hp['eve_esperados_2016']
data_frame_hp['rme_2017']=data_frame_hp['2017']/data_frame_hp['eve_esperados_2017']
data_frame_hp['rme_2018']=data_frame_hp['2018']/data_frame_hp['eve_esperados_2018']
data_frame_hp['rme_2019']=data_frame_hp['2019']/data_frame_hp['eve_esperados_2019']

In [123]:
data_frame_hp.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,...,rme_2010,rme_2011,rme_2012,rme_2013,rme_2014,rme_2015,rme_2016,rme_2017,rme_2018,rme_2019
0,0101,TERRON COLORADO,1.386998,51,67,55,69,67,100,107,...,7.904907,8.736978,7.616842,7.038078,7.286268,8.888670,7.208554,6.454793,4.662170,7.466483
1,0102,VISTA HERMOSA,1.148898,5,6,6,8,7,14,10,...,0.935602,0.944565,1.003131,0.985121,0.919015,1.502309,0.813315,0.445286,1.851437,6.009235
2,0196,SECTOR PATIO BONITO,0.198371,0,0,4,5,7,5,7,...,0.000000,0.000000,3.873207,3.565936,5.322638,3.107458,3.297318,2.149124,3.002417,2.900294
3,0199,AGUACATAL,1.107991,10,11,17,19,11,20,32,...,1.940289,1.795639,2.947141,2.426043,1.497486,2.225393,2.698697,2.231675,2.610918,2.077033
4,0201,SANTA RITA,0.181149,11,10,11,10,17,21,19,...,13.054498,9.984523,11.663955,7.809908,14.155337,14.292132,9.800740,3.294822,6.575723,9.528088


In [124]:
data_frame_reporte_hp = data_frame_hp[['COD_BARRIO', 
                                    'BARRIO_OFICIAL', 
                                    '2010',
                                    '2011',
                                    '2012',
                                    '2013',
                                    '2014',
                                    '2015',
                                    '2016',
                                    '2017',
                                    '2018',
                                    '2019',
                                    'rme_2010',
                                    'rme_2011',
                                    'rme_2012',
                                    'rme_2013',
                                    'rme_2014',
                                    'rme_2015',
                                    'rme_2016',
                                    'rme_2017',
                                    'rme_2018',
                                    'rme_2019']]
                                    
                                    
data_frame_reporte_hp

,COD_BARRIO,BARRIO_OFICIAL,2010,2011,2012,2013,2014,2015,2016,2017,...,rme_2010,rme_2011,rme_2012,rme_2013,rme_2014,rme_2015,rme_2016,rme_2017,rme_2018,rme_2019
0,0101,TERRON COLORADO,51,67,55,69,67,100,107,105,...,7.904907,8.736978,7.616842,7.038078,7.286268,8.888670,7.208554,6.454793,4.662170,7.466483
1,0102,VISTA HERMOSA,5,6,6,8,7,14,10,6,...,0.935602,0.944565,1.003131,0.985121,0.919015,1.502309,0.813315,0.445286,1.851437,6.009235
2,0196,SECTOR PATIO BONITO,0,0,4,5,7,5,7,5,...,0.000000,0.000000,3.873207,3.565936,5.322638,3.107458,3.297318,2.149124,3.002417,2.900294
3,0199,AGUACATAL,10,11,17,19,11,20,32,29,...,1.940289,1.795639,2.947141,2.426043,1.497486,2.225393,2.698697,2.231675,2.610918,2.077033
4,0201,SANTA RITA,11,10,11,10,17,21,19,7,...,13.054498,9.984523,11.663955,7.809908,14.155337,14.292132,9.800740,3.294822,6.575723,9.528088
5,0202,SANTA TERESITA,17,16,19,21,26,10,25,24,...,13.249971,10.491699,13.231384,10.771191,14.218152,4.469678,8.469228,7.418972,6.786356,8.343399
6,0203,ARBOLEDAS,2,3,5,1,9,8,5,8,...,1.085795,1.370247,2.425345,0.357270,3.428184,2.490680,1.179847,1.722559,0.859460,1.452897
7,0204,NORMANDIA,6,11,7,15,12,16,15,10,...,5.149959,7.943375,5.368293,8.472708,7.226659,7.875583,5.596050,3.404231,7.813189,9.188181
8,0205,JUANAMBU,6,7,6,13,18,9,19,13,...,5.832548,5.724861,5.211275,8.316276,12.276749,5.017182,8.027836,5.012067,15.004435,20.812013
9,0206,CENTENARIO,34,29,33,55,48,55,70,81,...,34.203630,24.544327,29.661485,36.411151,33.879603,31.729718,30.607588,32.318021,37.027484,29.614403


In [125]:
import datetime
now = datetime.datetime.now()

ruta = './reportes/'
nombre_base = 'densidad_ajustada'
fecha = str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'-'+str(now.hour).zfill(2)+str(now.minute).zfill(2)
nombre_archivo = ruta + nombre_base + fecha +'.xlsx'

writer = pd.ExcelWriter(nombre_archivo)
data_frame_reporte_ho.to_excel(writer, 'rme_homicidios')
data_frame_reporte_hp.to_excel(writer, 'rme_hurto_a_personas')
writer.save()

**3.3 LESIONES PERSONALES** 

In [126]:
mask_lp = (df_gen_resumen_conflictividad['CONFLICTIVIDAD']=='LESIONES PERSONALES')

df_gen_resumen_conflictividad_lp = pd.DataFrame() 
df_gen_resumen_conflictividad_lp = df_gen_resumen_conflictividad[mask_lp]

**3.3.1 Lesiones personales x barrio**

In [127]:
tabla_eventos_conflictividad_lp_x_barrio = pd.crosstab(index=[df_gen_resumen_conflictividad_lp.COD_BARRIO, 
                                                              df_gen_resumen_conflictividad_lp.BARRIO_OFICIAL, 
                                                              df_gen_resumen_conflictividad_lp.AREA_km2],
                                                  columns=df_gen_resumen_conflictividad_lp.AÑO,
                                                  values=df_gen_resumen_conflictividad_lp.CONFLICTIVIDAD,
                                                  aggfunc=len,
                                                  margins=False,
                                                  margins_name='Total')

tabla_eventos_conflictividad_lp_x_barrio = tabla_eventos_conflictividad_lp_x_barrio.fillna(0).astype(int)
tabla_eventos_conflictividad_lp_x_barrio

,,AÑO,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
COD_BARRIO,BARRIO_OFICIAL,AREA_km2,,,,,,,,,,
0101,TERRON COLORADO,1.386998,97,102,90,113,92,109,109,144,122,11
0102,VISTA HERMOSA,1.148898,4,6,6,8,16,15,21,8,20,8
0196,SECTOR PATIO BONITO,0.198371,3,2,2,0,2,1,9,12,20,1
0199,AGUACATAL,1.107991,18,31,25,25,32,19,40,19,72,8
0201,SANTA RITA,0.181149,1,4,0,7,3,1,5,2,3,3
0202,SANTA TERESITA,0.275827,2,1,5,4,5,3,3,6,11,1
0203,ARBOLEDAS,0.395990,0,1,0,0,4,1,1,0,3,2
0204,NORMANDIA,0.250467,3,4,3,8,7,0,10,6,12,0
0205,JUANAMBU,0.221154,2,3,6,2,4,4,2,7,8,3


In [128]:
data_frame_lp = pd.DataFrame(tabla_eventos_conflictividad_lp_x_barrio.to_records())

data_frame_lp.head()


,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0101,TERRON COLORADO,1.386998,97,102,90,113,92,109,109,144,122,11
1,0102,VISTA HERMOSA,1.148898,4,6,6,8,16,15,21,8,20,8
2,0196,SECTOR PATIO BONITO,0.198371,3,2,2,0,2,1,9,12,20,1
3,0199,AGUACATAL,1.107991,18,31,25,25,32,19,40,19,72,8
4,0201,SANTA RITA,0.181149,1,4,0,7,3,1,5,2,3,3


In [129]:
data_frame_lp['densidad_esp_2010']=data_frame_lp['2010']/data_frame_lp['AREA_km2']
data_frame_lp['densidad_esp_2011']=data_frame_lp['2011']/data_frame_lp['AREA_km2']
data_frame_lp['densidad_esp_2012']=data_frame_lp['2012']/data_frame_lp['AREA_km2']
data_frame_lp['densidad_esp_2013']=data_frame_lp['2013']/data_frame_lp['AREA_km2']
data_frame_lp['densidad_esp_2014']=data_frame_lp['2014']/data_frame_lp['AREA_km2']
data_frame_lp['densidad_esp_2015']=data_frame_lp['2015']/data_frame_lp['AREA_km2']
data_frame_lp['densidad_esp_2016']=data_frame_lp['2016']/data_frame_lp['AREA_km2']
data_frame_lp['densidad_esp_2017']=data_frame_lp['2017']/data_frame_lp['AREA_km2']
data_frame_lp['densidad_esp_2018']=data_frame_lp['2018']/data_frame_lp['AREA_km2']
data_frame_lp['densidad_esp_2019']=data_frame_lp['2019']/data_frame_lp['AREA_km2']



data_frame_lp['eve_esperados_2010']=(data_frame_ref.iloc[2]['densidad_ref_2010'])*data_frame_lp['AREA_km2']
data_frame_lp['eve_esperados_2011']=(data_frame_ref.iloc[2]['densidad_ref_2011'])*data_frame_lp['AREA_km2']
data_frame_lp['eve_esperados_2012']=(data_frame_ref.iloc[2]['densidad_ref_2012'])*data_frame_lp['AREA_km2']
data_frame_lp['eve_esperados_2013']=(data_frame_ref.iloc[2]['densidad_ref_2013'])*data_frame_lp['AREA_km2']
data_frame_lp['eve_esperados_2014']=(data_frame_ref.iloc[2]['densidad_ref_2014'])*data_frame_lp['AREA_km2']
data_frame_lp['eve_esperados_2015']=(data_frame_ref.iloc[2]['densidad_ref_2015'])*data_frame_lp['AREA_km2']
data_frame_lp['eve_esperados_2016']=(data_frame_ref.iloc[2]['densidad_ref_2016'])*data_frame_lp['AREA_km2']
data_frame_lp['eve_esperados_2017']=(data_frame_ref.iloc[2]['densidad_ref_2017'])*data_frame_lp['AREA_km2']
data_frame_lp['eve_esperados_2018']=(data_frame_ref.iloc[2]['densidad_ref_2018'])*data_frame_lp['AREA_km2']
data_frame_lp['eve_esperados_2019']=(data_frame_ref.iloc[2]['densidad_ref_2019'])*data_frame_lp['AREA_km2']



data_frame_lp['rme_2010']=data_frame_lp['2010']/data_frame_lp['eve_esperados_2010']
data_frame_lp['rme_2011']=data_frame_lp['2011']/data_frame_lp['eve_esperados_2011']
data_frame_lp['rme_2012']=data_frame_lp['2012']/data_frame_lp['eve_esperados_2012']
data_frame_lp['rme_2013']=data_frame_lp['2013']/data_frame_lp['eve_esperados_2013']
data_frame_lp['rme_2014']=data_frame_lp['2014']/data_frame_lp['eve_esperados_2014']
data_frame_lp['rme_2015']=data_frame_lp['2015']/data_frame_lp['eve_esperados_2015']
data_frame_lp['rme_2016']=data_frame_lp['2016']/data_frame_lp['eve_esperados_2016']
data_frame_lp['rme_2017']=data_frame_lp['2017']/data_frame_lp['eve_esperados_2017']
data_frame_lp['rme_2018']=data_frame_lp['2018']/data_frame_lp['eve_esperados_2018']
data_frame_lp['rme_2019']=data_frame_lp['2019']/data_frame_lp['eve_esperados_2019']


In [130]:
data_frame_lp.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,...,rme_2010,rme_2011,rme_2012,rme_2013,rme_2014,rme_2015,rme_2016,rme_2017,rme_2018,rme_2019
0,0101,TERRON COLORADO,1.386998,97,102,90,113,92,109,109,...,5.277683,4.898079,4.246916,5.022099,5.190431,7.043678,7.412083,10.124045,9.249168,12.607877
1,0102,VISTA HERMOSA,1.148898,4,6,6,8,16,15,21,...,0.262740,0.347833,0.341804,0.429231,1.089758,1.170196,1.723961,0.679010,1.830489,11.069644
2,0196,SECTOR PATIO BONITO,0.198371,3,2,2,0,2,1,9,...,1.141278,0.671512,0.659872,0.000000,0.788940,0.451826,4.279123,5.898904,10.601598,8.013971
3,0199,AGUACATAL,1.107991,18,31,25,25,32,19,40,...,1.225981,1.863490,1.476764,1.390870,2.259984,1.536974,3.404972,1.672188,6.833059,11.478340
4,0201,SANTA RITA,0.181149,1,4,0,7,3,1,5,...,0.416593,1.470708,0.000000,2.382022,1.295919,0.494782,2.603303,1.076620,1.741426,26.327612


In [131]:
data_frame_reporte_lp = data_frame_lp[['COD_BARRIO', 
                                    'BARRIO_OFICIAL', 
                                    '2010',
                                    '2011',
                                    '2012',
                                    '2013',
                                    '2014',
                                    '2015',
                                    '2016',
                                    '2017',
                                    '2018',
                                    '2019',
                                    'rme_2010',
                                    'rme_2011',
                                    'rme_2012',
                                    'rme_2013',
                                    'rme_2014',
                                    'rme_2015',
                                    'rme_2016',
                                    'rme_2017',
                                    'rme_2018',
                                    'rme_2019']]
                                    
                                    
data_frame_reporte_lp

,COD_BARRIO,BARRIO_OFICIAL,2010,2011,2012,2013,2014,2015,2016,2017,...,rme_2010,rme_2011,rme_2012,rme_2013,rme_2014,rme_2015,rme_2016,rme_2017,rme_2018,rme_2019
0,0101,TERRON COLORADO,97,102,90,113,92,109,109,144,...,5.277683,4.898079,4.246916,5.022099,5.190431,7.043678,7.412083,10.124045,9.249168,12.607877
1,0102,VISTA HERMOSA,4,6,6,8,16,15,21,8,...,0.262740,0.347833,0.341804,0.429231,1.089758,1.170196,1.723961,0.679010,1.830489,11.069644
2,0196,SECTOR PATIO BONITO,3,2,2,0,2,1,9,12,...,1.141278,0.671512,0.659872,0.000000,0.788940,0.451826,4.279123,5.898904,10.601598,8.013971
3,0199,AGUACATAL,18,31,25,25,32,19,40,19,...,1.225981,1.863490,1.476764,1.390870,2.259984,1.536974,3.404972,1.672188,6.833059,11.478340
4,0201,SANTA RITA,1,4,0,7,3,1,5,2,...,0.416593,1.470708,0.000000,2.382022,1.295919,0.494782,2.603303,1.076620,1.741426,26.327612
5,0202,SANTA TERESITA,2,1,5,4,5,3,3,6,...,0.547194,0.241471,1.186426,0.893936,1.418486,0.974841,1.025828,2.121203,4.193484,5.763532
6,0203,ARBOLEDAS,0,1,0,0,4,1,1,0,...,0.000000,0.168196,0.000000,0.000000,0.790436,0.226342,0.238180,0.000000,0.796628,8.029167
7,0204,NORMANDIA,3,4,3,8,7,0,10,6,...,0.903897,1.063681,0.783932,1.968896,2.186954,0.000000,3.765648,2.335978,5.037906,0.000000
8,0205,JUANAMBU,2,3,6,2,4,4,2,7,...,0.682468,0.903498,1.775673,0.557465,1.415325,1.621114,0.852951,3.086527,3.803762,21.565079
9,0206,CENTENARIO,6,3,3,10,18,19,12,14,...,2.118798,0.935004,0.918796,2.884520,6.591053,7.968807,5.296167,6.388315,3.444353,0.000000


In [132]:
import datetime
now = datetime.datetime.now()

ruta = './reportes/'
nombre_base = 'densidad_ajustada'
fecha = str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'-'+str(now.hour).zfill(2)+str(now.minute).zfill(2)
nombre_archivo = ruta + nombre_base + fecha +'.xlsx'

writer = pd.ExcelWriter(nombre_archivo)
data_frame_reporte_ho.to_excel(writer, 'rme_homicidios')
data_frame_reporte_hp.to_excel(writer, 'rme_hurto_a_personas')
data_frame_reporte_lp.to_excel(writer, 'rme_lesiones_a_personas')
writer.save()

In [133]:
data_frame_reporte_lp.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
2010,333.0,13.282282,16.632950,0.0,3.000000,8.000000,16.000000,97.000000
2011,333.0,14.165165,18.250921,0.0,3.000000,8.000000,16.000000,102.000000
2012,333.0,13.645646,17.248728,0.0,3.000000,8.000000,17.000000,102.000000
2013,333.0,17.516517,19.440298,0.0,5.000000,12.000000,22.000000,113.000000
2014,333.0,15.579580,16.983616,0.0,5.000000,10.000000,20.000000,100.000000
2015,333.0,16.456456,18.198423,0.0,4.000000,11.000000,20.000000,109.000000
2016,333.0,19.147147,21.923319,0.0,6.000000,12.000000,22.000000,123.000000
2017,333.0,20.654655,24.777157,0.0,6.000000,13.000000,24.000000,163.000000
2018,333.0,23.258258,26.228019,0.0,8.000000,14.000000,27.000000,159.000000
2019,333.0,3.471471,4.429154,0.0,1.000000,2.000000,5.000000,30.000000


**3.4 VIOLENCIA INTRAFAMILIAR** 

In [134]:
mask_vi = (df_gen_resumen_conflictividad['CONFLICTIVIDAD']=='VIOLENCIA INTRAFAMILIAR')

df_gen_resumen_conflictividad_vi= pd.DataFrame() 
df_gen_resumen_conflictividad_vi = df_gen_resumen_conflictividad[mask_vi]            

**3.4.3 Violencia Intrafamilioar territorio barrio **

In [135]:
tabla_eventos_conflictividad_vi_x_barrio = pd.crosstab(index=[df_gen_resumen_conflictividad_vi.COD_BARRIO, 
                                                              df_gen_resumen_conflictividad_vi.BARRIO_OFICIAL, 
                                                              df_gen_resumen_conflictividad_vi.AREA_km2],
                                                  columns=df_gen_resumen_conflictividad_vi.AÑO,
                                                  values=df_gen_resumen_conflictividad_vi.CONFLICTIVIDAD,
                                                  aggfunc=len,
                                                  margins=False,
                                                  margins_name='Total')

tabla_eventos_conflictividad_vi_x_barrio = tabla_eventos_conflictividad_vi_x_barrio.fillna(0).astype(int)
tabla_eventos_conflictividad_vi_x_barrio

,,AÑO,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
COD_BARRIO,BARRIO_OFICIAL,AREA_km2,,,,,,,,,,
0101,TERRON COLORADO,1.386998,34,79,89,73,47,90,75,103,76,10
0102,VISTA HERMOSA,1.148898,3,11,9,6,2,19,22,8,19,2
0196,SECTOR PATIO BONITO,0.198371,0,0,2,2,1,3,4,5,8,3
0199,AGUACATAL,1.107991,5,23,23,13,14,19,15,28,41,3
0201,SANTA RITA,0.181149,1,0,3,0,2,0,5,1,4,0
0202,SANTA TERESITA,0.275827,0,0,3,5,2,8,7,6,8,0
0203,ARBOLEDAS,0.395990,0,2,0,0,0,2,3,1,3,0
0204,NORMANDIA,0.250467,1,2,5,1,1,3,5,5,12,1
0205,JUANAMBU,0.221154,0,2,1,1,3,0,1,10,2,1


In [136]:
data_frame_vi = pd.DataFrame(tabla_eventos_conflictividad_vi_x_barrio.to_records())
data_frame_vi.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0101,TERRON COLORADO,1.386998,34,79,89,73,47,90,75,103,76,10
1,0102,VISTA HERMOSA,1.148898,3,11,9,6,2,19,22,8,19,2
2,0196,SECTOR PATIO BONITO,0.198371,0,0,2,2,1,3,4,5,8,3
3,0199,AGUACATAL,1.107991,5,23,23,13,14,19,15,28,41,3
4,0201,SANTA RITA,0.181149,1,0,3,0,2,0,5,1,4,0


In [137]:
data_frame_vi['densidad_esp_2010']=data_frame_vi['2010']/data_frame_vi['AREA_km2']
data_frame_vi['densidad_esp_2011']=data_frame_vi['2011']/data_frame_vi['AREA_km2']
data_frame_vi['densidad_esp_2012']=data_frame_vi['2012']/data_frame_vi['AREA_km2']
data_frame_vi['densidad_esp_2013']=data_frame_vi['2013']/data_frame_vi['AREA_km2']
data_frame_vi['densidad_esp_2014']=data_frame_vi['2014']/data_frame_vi['AREA_km2']
data_frame_vi['densidad_esp_2015']=data_frame_vi['2015']/data_frame_vi['AREA_km2']
data_frame_vi['densidad_esp_2016']=data_frame_vi['2016']/data_frame_vi['AREA_km2']
data_frame_vi['densidad_esp_2017']=data_frame_vi['2017']/data_frame_vi['AREA_km2']
data_frame_vi['densidad_esp_2018']=data_frame_vi['2018']/data_frame_vi['AREA_km2']
data_frame_vi['densidad_esp_2019']=data_frame_vi['2019']/data_frame_vi['AREA_km2']


data_frame_vi['eve_esperados_2010']=(data_frame_ref.iloc[3]['densidad_ref_2010'])*data_frame_vi['AREA_km2']
data_frame_vi['eve_esperados_2011']=(data_frame_ref.iloc[3]['densidad_ref_2011'])*data_frame_vi['AREA_km2']
data_frame_vi['eve_esperados_2012']=(data_frame_ref.iloc[3]['densidad_ref_2012'])*data_frame_vi['AREA_km2']
data_frame_vi['eve_esperados_2013']=(data_frame_ref.iloc[3]['densidad_ref_2013'])*data_frame_vi['AREA_km2']
data_frame_vi['eve_esperados_2014']=(data_frame_ref.iloc[3]['densidad_ref_2014'])*data_frame_vi['AREA_km2']
data_frame_vi['eve_esperados_2015']=(data_frame_ref.iloc[3]['densidad_ref_2015'])*data_frame_vi['AREA_km2']
data_frame_vi['eve_esperados_2016']=(data_frame_ref.iloc[3]['densidad_ref_2016'])*data_frame_vi['AREA_km2']
data_frame_vi['eve_esperados_2017']=(data_frame_ref.iloc[3]['densidad_ref_2017'])*data_frame_vi['AREA_km2']
data_frame_vi['eve_esperados_2018']=(data_frame_ref.iloc[3]['densidad_ref_2018'])*data_frame_vi['AREA_km2']
data_frame_vi['eve_esperados_2019']=(data_frame_ref.iloc[3]['densidad_ref_2019'])*data_frame_vi['AREA_km2']



data_frame_vi['rme_2010']=data_frame_vi['2010']/data_frame_vi['eve_esperados_2010']
data_frame_vi['rme_2011']=data_frame_vi['2011']/data_frame_vi['eve_esperados_2011']
data_frame_vi['rme_2012']=data_frame_vi['2012']/data_frame_vi['eve_esperados_2012']
data_frame_vi['rme_2013']=data_frame_vi['2013']/data_frame_vi['eve_esperados_2013']
data_frame_vi['rme_2014']=data_frame_vi['2014']/data_frame_vi['eve_esperados_2014']
data_frame_vi['rme_2015']=data_frame_vi['2015']/data_frame_vi['eve_esperados_2015']
data_frame_vi['rme_2016']=data_frame_vi['2016']/data_frame_vi['eve_esperados_2016']
data_frame_vi['rme_2017']=data_frame_vi['2017']/data_frame_vi['eve_esperados_2017']
data_frame_vi['rme_2018']=data_frame_vi['2018']/data_frame_vi['eve_esperados_2018']
data_frame_vi['rme_2019']=data_frame_vi['2019']/data_frame_vi['eve_esperados_2019']

In [138]:
data_frame_lp.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,...,rme_2010,rme_2011,rme_2012,rme_2013,rme_2014,rme_2015,rme_2016,rme_2017,rme_2018,rme_2019
0,0101,TERRON COLORADO,1.386998,97,102,90,113,92,109,109,...,5.277683,4.898079,4.246916,5.022099,5.190431,7.043678,7.412083,10.124045,9.249168,12.607877
1,0102,VISTA HERMOSA,1.148898,4,6,6,8,16,15,21,...,0.262740,0.347833,0.341804,0.429231,1.089758,1.170196,1.723961,0.679010,1.830489,11.069644
2,0196,SECTOR PATIO BONITO,0.198371,3,2,2,0,2,1,9,...,1.141278,0.671512,0.659872,0.000000,0.788940,0.451826,4.279123,5.898904,10.601598,8.013971
3,0199,AGUACATAL,1.107991,18,31,25,25,32,19,40,...,1.225981,1.863490,1.476764,1.390870,2.259984,1.536974,3.404972,1.672188,6.833059,11.478340
4,0201,SANTA RITA,0.181149,1,4,0,7,3,1,5,...,0.416593,1.470708,0.000000,2.382022,1.295919,0.494782,2.603303,1.076620,1.741426,26.327612


In [139]:
data_frame_reporte_vi = data_frame_vi[['COD_BARRIO', 
                                    'BARRIO_OFICIAL', 
                                    '2010',
                                    '2011',
                                    '2012',
                                    '2013',
                                    '2014',
                                    '2015',
                                    '2016',
                                    '2017',
                                    '2018',
                                    '2019',
                                    'rme_2010',
                                    'rme_2011',
                                    'rme_2012',
                                    'rme_2013',
                                    'rme_2014',
                                    'rme_2015',
                                    'rme_2016',
                                    'rme_2017',
                                    'rme_2018',
                                    'rme_2019']]
                                    
                                    
data_frame_reporte_vi

,COD_BARRIO,BARRIO_OFICIAL,2010,2011,2012,2013,2014,2015,2016,2017,...,rme_2010,rme_2011,rme_2012,rme_2013,rme_2014,rme_2015,rme_2016,rme_2017,rme_2018,rme_2019
0,0101,TERRON COLORADO,34,79,89,73,47,90,75,103,...,0.454389,0.926317,1.034245,0.684642,0.448425,0.906651,0.630798,0.650821,0.420843,0.326129
1,0102,VISTA HERMOSA,3,11,9,6,2,19,22,8,...,0.048402,0.155711,0.126261,0.067934,0.023036,0.231071,0.223381,0.061025,0.127015,0.078743
2,0196,SECTOR PATIO BONITO,0,0,2,2,1,3,4,5,...,0.000000,0.000000,0.162503,0.131150,0.066710,0.211309,0.235227,0.220899,0.309738,0.684083
3,0199,AGUACATAL,5,23,23,13,14,19,15,28,...,0.083649,0.337598,0.334581,0.152624,0.167209,0.239602,0.157928,0.221474,0.284204,0.122476
4,0201,SANTA RITA,1,0,3,0,2,0,5,1,...,0.102327,0.000000,0.266929,0.000000,0.146104,0.000000,0.321988,0.048380,0.169593,0.000000
5,0202,SANTA TERESITA,0,0,3,5,2,8,7,6,...,0.000000,0.000000,0.175305,0.235803,0.095954,0.405254,0.296051,0.190641,0.222759,0.000000
6,0203,ARBOLEDAS,0,2,0,0,0,2,3,1,...,0.000000,0.082140,0.000000,0.000000,0.000000,0.070570,0.088377,0.022132,0.058186,0.000000
7,0204,NORMANDIA,1,2,5,1,1,3,5,5,...,0.074007,0.129864,0.321758,0.051936,0.052835,0.167357,0.232876,0.174953,0.367971,0.180599
8,0205,JUANAMBU,0,2,1,1,3,0,1,10,...,0.000000,0.147076,0.072881,0.058819,0.179512,0.000000,0.052748,0.396283,0.069457,0.204536
9,0206,CENTENARIO,0,2,1,1,0,6,6,5,...,0.000000,0.152205,0.075422,0.060871,0.000000,0.392297,0.327527,0.205051,0.107819,0.211668


In [140]:
import datetime
now = datetime.datetime.now()

ruta = './reportes/'
nombre_base = 'densidad_ajustada'
fecha = str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'-'+str(now.hour).zfill(2)+str(now.minute).zfill(2)
nombre_archivo = ruta + nombre_base + fecha +'.xlsx'

writer = pd.ExcelWriter(nombre_archivo)
data_frame_reporte_ho.to_excel(writer, 'rme_homicidios')
data_frame_reporte_hp.to_excel(writer, 'rme_hurto_a_personas')
data_frame_reporte_lp.to_excel(writer, 'rme_lesiones_a_personas')
data_frame_reporte_vi.to_excel(writer, 'rme_violencia_intrafamiliar')
writer.save()

**3.5 DAV0 (HP+LP+VI)** 

In [141]:
mask_dav0 = ((df_gen_resumen_conflictividad['CONFLICTIVIDAD']=='HURTO PERSONA') |
           (df_gen_resumen_conflictividad['CONFLICTIVIDAD']=='LESIONES PERSONALES') |
           (df_gen_resumen_conflictividad['CONFLICTIVIDAD']=='VIOLENCIA INTRAFAMILIAR'))

df_gen_resumen_conflictividad_dav0= pd.DataFrame() 
df_gen_resumen_conflictividad_dav0 = df_gen_resumen_conflictividad[mask_dav0]
df_gen_resumen_conflictividad_dav0.conflictividad = 'DAV0'

**3.5.3 DAV0 territorio barrio **

In [142]:
tabla_eventos_conflictividad_dav0_x_barrio = pd.crosstab(index=[df_gen_resumen_conflictividad_dav0.COD_BARRIO, 
                                                                df_gen_resumen_conflictividad_dav0.BARRIO_OFICIAL, 
                                                                df_gen_resumen_conflictividad_dav0.AREA_km2],
                                                  columns=df_gen_resumen_conflictividad_dav0.AÑO,
                                                  values=df_gen_resumen_conflictividad_dav0.CONFLICTIVIDAD,
                                                  aggfunc=len,
                                                  margins=False,
                                                  margins_name='Total')

tabla_eventos_conflictividad_dav0_x_barrio = tabla_eventos_conflictividad_dav0_x_barrio.fillna(0).astype(int)
tabla_eventos_conflictividad_dav0_x_barrio

,,AÑO,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
COD_BARRIO,BARRIO_OFICIAL,AREA_km2,,,,,,,,,,
0101,TERRON COLORADO,1.386998,182,248,234,255,206,299,291,352,274,39
0102,VISTA HERMOSA,1.148898,12,23,21,22,25,48,53,22,64,22
0196,SECTOR PATIO BONITO,0.198371,3,2,8,7,10,9,20,22,35,5
0199,AGUACATAL,1.107991,33,65,65,57,57,58,87,76,147,15
0201,SANTA RITA,0.181149,13,14,14,17,22,22,29,10,21,6
0202,SANTA TERESITA,0.275827,19,17,27,30,33,21,35,36,41,5
0203,ARBOLEDAS,0.395990,2,6,5,1,13,11,9,9,10,3
0204,NORMANDIA,0.250467,10,17,15,24,20,19,30,21,47,5
0205,JUANAMBU,0.221154,8,12,13,16,25,13,22,30,49,12


In [143]:
data_frame_dav0 = pd.DataFrame(tabla_eventos_conflictividad_dav0_x_barrio.to_records())
data_frame_dav0.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0101,TERRON COLORADO,1.386998,182,248,234,255,206,299,291,352,274,39
1,0102,VISTA HERMOSA,1.148898,12,23,21,22,25,48,53,22,64,22
2,0196,SECTOR PATIO BONITO,0.198371,3,2,8,7,10,9,20,22,35,5
3,0199,AGUACATAL,1.107991,33,65,65,57,57,58,87,76,147,15
4,0201,SANTA RITA,0.181149,13,14,14,17,22,22,29,10,21,6


In [144]:
data_frame_dav0['densidad_esp_2010']=data_frame_dav0['2010']/data_frame_dav0['AREA_km2']
data_frame_dav0['densidad_esp_2011']=data_frame_dav0['2011']/data_frame_dav0['AREA_km2']
data_frame_dav0['densidad_esp_2012']=data_frame_dav0['2012']/data_frame_dav0['AREA_km2']
data_frame_dav0['densidad_esp_2013']=data_frame_dav0['2013']/data_frame_dav0['AREA_km2']
data_frame_dav0['densidad_esp_2014']=data_frame_dav0['2014']/data_frame_dav0['AREA_km2']
data_frame_dav0['densidad_esp_2015']=data_frame_dav0['2015']/data_frame_dav0['AREA_km2']
data_frame_dav0['densidad_esp_2016']=data_frame_dav0['2016']/data_frame_dav0['AREA_km2']
data_frame_dav0['densidad_esp_2017']=data_frame_dav0['2017']/data_frame_dav0['AREA_km2']
data_frame_dav0['densidad_esp_2018']=data_frame_dav0['2018']/data_frame_dav0['AREA_km2']
data_frame_dav0['densidad_esp_2019']=data_frame_dav0['2019']/data_frame_dav0['AREA_km2']


data_frame_dav0['eve_esperados_2010']=(data_frame_ref.iloc[1]['densidad_ref_2010']+
                                       data_frame_ref.iloc[2]['densidad_ref_2010']+
                                       data_frame_ref.iloc[3]['densidad_ref_2010'])*data_frame_dav0['AREA_km2']
data_frame_dav0['eve_esperados_2011']=(data_frame_ref.iloc[1]['densidad_ref_2011']+
                                       data_frame_ref.iloc[2]['densidad_ref_2011']+
                                       data_frame_ref.iloc[3]['densidad_ref_2011'])*data_frame_dav0['AREA_km2']
data_frame_dav0['eve_esperados_2012']=(data_frame_ref.iloc[1]['densidad_ref_2012']+
                                       data_frame_ref.iloc[2]['densidad_ref_2012']+
                                       data_frame_ref.iloc[3]['densidad_ref_2012'])*data_frame_dav0['AREA_km2']
data_frame_dav0['eve_esperados_2013']=(data_frame_ref.iloc[1]['densidad_ref_2013']+
                                       data_frame_ref.iloc[2]['densidad_ref_2013']+
                                       data_frame_ref.iloc[3]['densidad_ref_2013'])*data_frame_dav0['AREA_km2']
data_frame_dav0['eve_esperados_2014']=(data_frame_ref.iloc[1]['densidad_ref_2014']+
                                       data_frame_ref.iloc[2]['densidad_ref_2014']+
                                       data_frame_ref.iloc[3]['densidad_ref_2014'])*data_frame_dav0['AREA_km2']
data_frame_dav0['eve_esperados_2015']=(data_frame_ref.iloc[1]['densidad_ref_2015']+
                                       data_frame_ref.iloc[2]['densidad_ref_2015']+
                                       data_frame_ref.iloc[3]['densidad_ref_2015'])*data_frame_dav0['AREA_km2']
data_frame_dav0['eve_esperados_2016']=(data_frame_ref.iloc[1]['densidad_ref_2016']+
                                       data_frame_ref.iloc[2]['densidad_ref_2016']+
                                       data_frame_ref.iloc[3]['densidad_ref_2016'])*data_frame_dav0['AREA_km2']
data_frame_dav0['eve_esperados_2017']=(data_frame_ref.iloc[1]['densidad_ref_2017']+
                                       data_frame_ref.iloc[2]['densidad_ref_2017']+
                                       data_frame_ref.iloc[3]['densidad_ref_2017'])*data_frame_dav0['AREA_km2']
data_frame_dav0['eve_esperados_2018']=(data_frame_ref.iloc[1]['densidad_ref_2018']+
                                       data_frame_ref.iloc[2]['densidad_ref_2018']+
                                       data_frame_ref.iloc[3]['densidad_ref_2018'])*data_frame_dav0['AREA_km2']
data_frame_dav0['eve_esperados_2019']=(data_frame_ref.iloc[1]['densidad_ref_2019']+
                                       data_frame_ref.iloc[2]['densidad_ref_2019']+
                                       data_frame_ref.iloc[3]['densidad_ref_2019'])*data_frame_dav0['AREA_km2']


data_frame_dav0['rme_2010']=data_frame_dav0['2010']/data_frame_dav0['eve_esperados_2010']
data_frame_dav0['rme_2011']=data_frame_dav0['2011']/data_frame_dav0['eve_esperados_2011']
data_frame_dav0['rme_2012']=data_frame_dav0['2012']/data_frame_dav0['eve_esperados_2012']
data_frame_dav0['rme_2013']=data_frame_dav0['2013']/data_frame_dav0['eve_esperados_2013']
data_frame_dav0['rme_2014']=data_frame_dav0['2014']/data_frame_dav0['eve_esperados_2014']
data_frame_dav0['rme_2015']=data_frame_dav0['2015']/data_frame_dav0['eve_esperados_2015']
data_frame_dav0['rme_2016']=data_frame_dav0['2016']/data_frame_dav0['eve_esperados_2016']
data_frame_dav0['rme_2017']=data_frame_dav0['2017']/data_frame_dav0['eve_esperados_2017']
data_frame_dav0['rme_2018']=data_frame_dav0['2018']/data_frame_dav0['eve_esperados_2018']
data_frame_dav0['rme_2019']=data_frame_dav0['2019']/data_frame_dav0['eve_esperados_2019']

In [147]:
data_frame_reporte_dav0 = data_frame_dav0[columnas_]
                                          
data_frame_reporte_dav0

,COD_BARRIO,BARRIO_OFICIAL,2010,2011,2012,2013,2014,2015,2016,2017,...,rme_2010,rme_2011,rme_2012,rme_2013,rme_2014,rme_2015,rme_2016,rme_2017,rme_2018,rme_2019
0,0101,TERRON COLORADO,182,248,234,255,206,299,291,352,...,1.826268,2.179702,2.044278,1.835464,1.563786,2.373174,1.960305,1.864880,1.304255,1.148884
1,0102,VISTA HERMOSA,12,23,21,22,25,48,53,22,...,0.145368,0.244044,0.221482,0.191171,0.229110,0.459932,0.431024,0.140710,0.367778,0.782400
2,0196,SECTOR PATIO BONITO,3,2,8,7,10,9,20,22,...,0.210481,0.122906,0.488667,0.352292,0.530773,0.499458,0.942018,0.814947,1.164872,1.029864
3,0199,AGUACATAL,33,65,65,57,57,58,87,76,...,0.414521,0.715152,0.710849,0.513595,0.541658,0.576270,0.733652,0.504036,0.875929,0.553150
4,0201,SANTA RITA,13,14,14,17,22,22,29,10,...,0.998797,0.942138,0.936468,0.936905,1.278715,1.336971,1.495787,0.405648,0.765371,1.353330
5,0202,SANTA TERESITA,19,17,27,30,33,21,35,36,...,0.958708,0.751335,1.186117,1.085841,1.259690,0.838141,1.185600,0.959070,0.981375,0.740664
6,0203,ARBOLEDAS,2,6,5,1,13,11,9,9,...,0.070293,0.184709,0.152998,0.025211,0.345657,0.305803,0.212356,0.167010,0.166726,0.309545
7,0204,NORMANDIA,10,17,15,24,20,19,30,21,...,0.555673,0.827409,0.725674,0.956628,0.840749,0.835099,1.119123,0.616103,1.238898,0.815657
8,0205,JUANAMBU,8,12,13,16,25,13,22,30,...,0.503458,0.661465,0.712276,0.722281,1.190230,0.647116,0.929467,0.996805,1.462812,2.217039
9,0206,CENTENARIO,40,34,37,66,66,80,88,100,...,2.605073,1.939506,2.097938,3.083304,3.251778,4.121118,3.847514,3.438547,3.182114,2.294349


In [149]:
import datetime
now = datetime.datetime.now()

ruta = './reportes/'
nombre_base = 'densidad_ajustada'
fecha = str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'-'+str(now.hour).zfill(2)+str(now.minute).zfill(2)
nombre_archivo = ruta + nombre_base + fecha +'.xlsx'

writer = pd.ExcelWriter(nombre_archivo)
data_frame_reporte_ho.to_excel(writer, 'rme_homicidios')
data_frame_reporte_hp.to_excel(writer, 'rme_hurto_a_personas')
data_frame_reporte_lp.to_excel(writer, 'rme_lesiones_a_personas')
data_frame_reporte_vi.to_excel(writer, 'rme_violencia_intrafamiliar')
data_frame_reporte_dav0.to_excel(writer, 'rme_dav0_hp_lp_vi')
writer.save()

**3.6 DAV1 (HP+LP+VI+HO)** 

In [150]:
mask_dav1 = ((df_gen_resumen_conflictividad['CONFLICTIVIDAD']=='HURTO PERSONA') |
            (df_gen_resumen_conflictividad['CONFLICTIVIDAD']=='LESIONES PERSONALES') |
            (df_gen_resumen_conflictividad['CONFLICTIVIDAD']=='VIOLENCIA INTARAFAMILIAR') |
            (df_gen_resumen_conflictividad['CONFLICTIVIDAD']=='HOMICIDIO')
            )

df_gen_resumen_conflictividad_dav1= pd.DataFrame() 
df_gen_resumen_conflictividad_dav1 = df_gen_resumen_conflictividad[mask_dav1]
df_gen_resumen_conflictividad_dav1.CONFLICTIVIDAD = 'DAV1'

C:\Anaconda\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


**3.6.3 DAV1 territorio barrio **

In [155]:
tabla_eventos_conflictividad_dav1_x_barrio = pd.crosstab(index=[df_gen_resumen_conflictividad_dav1.COD_BARRIO, 
                                                                df_gen_resumen_conflictividad_dav1.BARRIO_OFICIAL,
                                                                df_gen_resumen_conflictividad_dav1.AREA_km2],
                                                  columns=df_gen_resumen_conflictividad_dav1.AÑO,
                                                  values=df_gen_resumen_conflictividad_dav1.CONFLICTIVIDAD,
                                                  aggfunc=len,
                                                  margins=False,
                                                  margins_name='Total'
                                                  )
tabla_eventos_conflictividad_dav1_x_barrio = tabla_eventos_conflictividad_dav1_x_barrio.fillna(0).astype(int)
tabla_eventos_conflictividad_dav1_x_barrio.head()

,,AÑO,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
COD_BARRIO,BARRIO_OFICIAL,AREA_km2,,,,,,,,,,
0101,TERRON COLORADO,1.386998,185,205,180,214,189,226,235,256,200,29
0102,VISTA HERMOSA,1.148898,9,12,12,21,23,35,34,17,50,20
0196,SECTOR PATIO BONITO,0.198371,3,2,6,5,9,6,16,18,28,2
0199,AGUACATAL,1.107991,28,51,45,61,48,41,75,51,107,12
0201,SANTA RITA,0.181149,13,16,12,17,22,22,24,9,17,6


In [156]:
data_frame_dav1 = pd.DataFrame(tabla_eventos_conflictividad_dav1_x_barrio.to_records())
data_frame_dav1.head()

,COD_BARRIO,BARRIO_OFICIAL,AREA_km2,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0101,TERRON COLORADO,1.386998,185,205,180,214,189,226,235,256,200,29
1,0102,VISTA HERMOSA,1.148898,9,12,12,21,23,35,34,17,50,20
2,0196,SECTOR PATIO BONITO,0.198371,3,2,6,5,9,6,16,18,28,2
3,0199,AGUACATAL,1.107991,28,51,45,61,48,41,75,51,107,12
4,0201,SANTA RITA,0.181149,13,16,12,17,22,22,24,9,17,6


In [157]:
data_frame_dav1['densidad_esp_2010']=data_frame_dav1['2010']/data_frame_dav1['AREA_km2']
data_frame_dav1['densidad_esp_2011']=data_frame_dav1['2011']/data_frame_dav1['AREA_km2']
data_frame_dav1['densidad_esp_2012']=data_frame_dav1['2012']/data_frame_dav1['AREA_km2']
data_frame_dav1['densidad_esp_2013']=data_frame_dav1['2013']/data_frame_dav1['AREA_km2']
data_frame_dav1['densidad_esp_2014']=data_frame_dav1['2014']/data_frame_dav1['AREA_km2']
data_frame_dav1['densidad_esp_2015']=data_frame_dav1['2015']/data_frame_dav1['AREA_km2']
data_frame_dav1['densidad_esp_2016']=data_frame_dav1['2016']/data_frame_dav1['AREA_km2']
data_frame_dav1['densidad_esp_2017']=data_frame_dav1['2017']/data_frame_dav1['AREA_km2']
data_frame_dav1['densidad_esp_2018']=data_frame_dav1['2018']/data_frame_dav1['AREA_km2']
data_frame_dav1['densidad_esp_2019']=data_frame_dav1['2019']/data_frame_dav1['AREA_km2']



data_frame_dav1['eve_esperados_2010']=(data_frame_ref.iloc[0]['densidad_ref_2010']+
                                       data_frame_ref.iloc[1]['densidad_ref_2010']+
                                       data_frame_ref.iloc[2]['densidad_ref_2010']+
                                       data_frame_ref.iloc[3]['densidad_ref_2010'])*data_frame_dav1['AREA_km2']
data_frame_dav1['eve_esperados_2011']=(data_frame_ref.iloc[0]['densidad_ref_2011']+
                                       data_frame_ref.iloc[1]['densidad_ref_2011']+
                                       data_frame_ref.iloc[2]['densidad_ref_2011']+
                                       data_frame_ref.iloc[3]['densidad_ref_2011'])*data_frame_dav1['AREA_km2']
data_frame_dav1['eve_esperados_2012']=(data_frame_ref.iloc[0]['densidad_ref_2012']+
                                       data_frame_ref.iloc[1]['densidad_ref_2012']+
                                       data_frame_ref.iloc[2]['densidad_ref_2012']+
                                       data_frame_ref.iloc[3]['densidad_ref_2012'])*data_frame_dav1['AREA_km2']
data_frame_dav1['eve_esperados_2013']=(data_frame_ref.iloc[0]['densidad_ref_2013']+
                                       data_frame_ref.iloc[1]['densidad_ref_2013']+
                                       data_frame_ref.iloc[2]['densidad_ref_2013']+
                                       data_frame_ref.iloc[3]['densidad_ref_2013'])*data_frame_dav1['AREA_km2']
data_frame_dav1['eve_esperados_2014']=(data_frame_ref.iloc[0]['densidad_ref_2014']+
                                       data_frame_ref.iloc[1]['densidad_ref_2014']+
                                       data_frame_ref.iloc[2]['densidad_ref_2014']+
                                       data_frame_ref.iloc[3]['densidad_ref_2014'])*data_frame_dav1['AREA_km2']
data_frame_dav1['eve_esperados_2015']=(data_frame_ref.iloc[0]['densidad_ref_2015']+
                                       data_frame_ref.iloc[1]['densidad_ref_2015']+
                                       data_frame_ref.iloc[2]['densidad_ref_2015']+
                                       data_frame_ref.iloc[3]['densidad_ref_2015'])*data_frame_dav1['AREA_km2']
data_frame_dav1['eve_esperados_2016']=(data_frame_ref.iloc[0]['densidad_ref_2016']+
                                       data_frame_ref.iloc[1]['densidad_ref_2016']+
                                       data_frame_ref.iloc[2]['densidad_ref_2016']+
                                       data_frame_ref.iloc[3]['densidad_ref_2016'])*data_frame_dav1['AREA_km2']
data_frame_dav1['eve_esperados_2017']=(data_frame_ref.iloc[0]['densidad_ref_2017']+
                                       data_frame_ref.iloc[1]['densidad_ref_2017']+
                                       data_frame_ref.iloc[2]['densidad_ref_2017']+
                                       data_frame_ref.iloc[3]['densidad_ref_2017'])*data_frame_dav1['AREA_km2']
data_frame_dav1['eve_esperados_2018']=(data_frame_ref.iloc[0]['densidad_ref_2018']+
                                       data_frame_ref.iloc[1]['densidad_ref_2016']+
                                       data_frame_ref.iloc[2]['densidad_ref_2018']+
                                       data_frame_ref.iloc[3]['densidad_ref_2018'])*data_frame_dav1['AREA_km2']
data_frame_dav1['eve_esperados_2019']=(data_frame_ref.iloc[0]['densidad_ref_2019']+
                                       data_frame_ref.iloc[1]['densidad_ref_2019']+
                                       data_frame_ref.iloc[2]['densidad_ref_2019']+
                                       data_frame_ref.iloc[3]['densidad_ref_2019'])*data_frame_dav1['AREA_km2']


data_frame_dav1['rme_2010']=data_frame_dav1['2010']/data_frame_dav1['eve_esperados_2010']
data_frame_dav1['rme_2011']=data_frame_dav1['2011']/data_frame_dav1['eve_esperados_2011']
data_frame_dav1['rme_2012']=data_frame_dav1['2012']/data_frame_dav1['eve_esperados_2012']
data_frame_dav1['rme_2013']=data_frame_dav1['2013']/data_frame_dav1['eve_esperados_2013']
data_frame_dav1['rme_2014']=data_frame_dav1['2014']/data_frame_dav1['eve_esperados_2014']
data_frame_dav1['rme_2015']=data_frame_dav1['2015']/data_frame_dav1['eve_esperados_2015']
data_frame_dav1['rme_2016']=data_frame_dav1['2016']/data_frame_dav1['eve_esperados_2016']
data_frame_dav1['rme_2017']=data_frame_dav1['2017']/data_frame_dav1['eve_esperados_2017']
data_frame_dav1['rme_2018']=data_frame_dav1['2018']/data_frame_dav1['eve_esperados_2018']
data_frame_dav1['rme_2019']=data_frame_dav1['2019']/data_frame_dav1['eve_esperados_2019']

In [161]:
data_frame_reporte_dav1 = data_frame_dav1[columnas_]
                                    
                                    
data_frame_reporte_dav1.head()

,COD_BARRIO,BARRIO_OFICIAL,2010,2011,2012,2013,2014,2015,2016,2017,...,rme_2010,rme_2011,rme_2012,rme_2013,rme_2014,rme_2015,rme_2016,rme_2017,rme_2018,rme_2019
0,0101,TERRON COLORADO,185,205,180,214,189,226,235,256,...,1.698836,1.644474,1.439818,1.469324,1.290352,1.492881,1.389628,1.229728,0.825286,0.808113
1,0102,VISTA HERMOSA,9,12,12,21,23,35,34,17,...,0.099774,0.116211,0.115881,0.174067,0.189569,0.279113,0.242719,0.098585,0.249080,0.672819
2,0196,SECTOR PATIO BONITO,3,2,6,5,9,6,16,18,...,0.192619,0.112176,0.335571,0.240034,0.429623,0.277119,0.661529,0.604561,0.807851,0.389675
3,0199,AGUACATAL,28,51,45,61,48,41,75,51,...,0.321868,0.512133,0.450596,0.524292,0.410230,0.339032,0.555177,0.306676,0.552711,0.418596
4,0201,SANTA RITA,13,16,12,17,22,22,24,9,...,0.914038,0.982729,0.734949,0.893704,1.150032,1.112706,1.086633,0.331019,0.537112,1.280165


In [159]:
import datetime
now = datetime.datetime.now()

ruta = './reportes/'
nombre_base = 'densidad_ajustada'
fecha = str(now.year)+ str(now.month).zfill(2)+ str(now.day).zfill(2)+'-'+str(now.hour).zfill(2)+str(now.minute).zfill(2)
nombre_archivo = ruta + nombre_base + fecha +'.xlsx'

writer = pd.ExcelWriter(nombre_archivo)
data_frame_reporte_ho.to_excel(writer, 'rme_homicidios')
data_frame_reporte_hp.to_excel(writer, 'rme_hurto_a_personas')
data_frame_reporte_lp.to_excel(writer, 'rme_lesiones_a_personas')
data_frame_reporte_vi.to_excel(writer, 'rme_violencia_intrafamiliar')
data_frame_reporte_dav0.to_excel(writer, 'rme_dav0_hp_lp_vi')
data_frame_reporte_dav1.to_excel(writer, 'rme_dav1_hp_lp_vi_ho')
writer.save()

In [185]:
my_columnas_ = data_frame_reporte_ho.columns
dictOfWords = {}

dictOfWords = { i : my_columnas_[i] for i in range(0, len(my_columnas_) ) }

In [186]:
dictOfWords

{0: 'COD_BARRIO',
 1: 'BARRIO_OFICIAL',
 2: '2010',
 3: '2011',
 4: '2012',
 5: '2013',
 6: '2014',
 7: '2015',
 8: '2016',
 9: '2017',
 10: '2018',
 11: '2019',
 12: 'rme_2010',
 13: 'rme_2011',
 14: 'rme_2012',
 15: 'rme_2013',
 16: 'rme_2014',
 17: 'rme_2015',
 18: 'rme_2016',
 19: 'rme_2017',
 20: 'rme_2018',
 21: 'rme_2019'}

In [215]:
quantiles = [.1, .2, .3, .4, .5, .6, .7, .8, .9, 1]

df_quantiles_ho = pd.DataFrame()
df_quantiles_ho['2015'] = data_frame_reporte_ho.rme_2015.quantile(quantiles)
df_quantiles_ho['2016'] = data_frame_reporte_ho.rme_2016.quantile(quantiles)
df_quantiles_ho['2017'] = data_frame_reporte_ho.rme_2017.quantile(quantiles)
df_quantiles_ho['2018'] = data_frame_reporte_ho.rme_2018.quantile(quantiles)

df_quantiles_hp = pd.DataFrame()
df_quantiles_hp['2015'] = data_frame_reporte_hp.rme_2015.quantile(quantiles)
df_quantiles_hp['2016'] = data_frame_reporte_hp.rme_2016.quantile(quantiles)
df_quantiles_hp['2017'] = data_frame_reporte_hp.rme_2017.quantile(quantiles)
df_quantiles_hp['2018'] = data_frame_reporte_hp.rme_2018.quantile(quantiles)

df_quantiles_lp = pd.DataFrame()
df_quantiles_lp['2015'] = data_frame_reporte_lp.rme_2015.quantile(quantiles)
df_quantiles_lp['2016'] = data_frame_reporte_lp.rme_2016.quantile(quantiles)
df_quantiles_lp['2017'] = data_frame_reporte_lp.rme_2017.quantile(quantiles)
df_quantiles_lp['2018'] = data_frame_reporte_lp.rme_2018.quantile(quantiles)

df_quantiles_vi = pd.DataFrame()
df_quantiles_vi['2015'] = data_frame_reporte_vi.rme_2015.quantile(quantiles)
df_quantiles_vi['2016'] = data_frame_reporte_vi.rme_2016.quantile(quantiles)
df_quantiles_vi['2017'] = data_frame_reporte_vi.rme_2017.quantile(quantiles)
df_quantiles_vi['2018'] = data_frame_reporte_vi.rme_2018.quantile(quantiles)


In [216]:
df_quantiles_ho = df_quantiles_ho.T
df_quantiles_hp = df_quantiles_hp.T
df_quantiles_lp = df_quantiles_lp.T
df_quantiles_vi = df_quantiles_vi.T


In [220]:
df_quantiles_hp

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
2015,2.205284,4.198852,5.370628,7.070287,8.305060,9.595600,11.866413,15.448887,21.793777,105.188184
2016,2.637327,4.292414,5.379031,6.311447,7.660230,8.951299,10.710525,13.024702,19.197124,89.564474
2017,2.637902,4.253285,6.062564,7.753112,9.233325,11.053597,12.611549,15.956896,21.444100,100.604530
2018,3.565242,5.262087,6.721469,7.804633,9.586149,11.666307,13.999011,17.765153,23.356884,136.229857


______

In [160]:


for delito in lista_delitos:
    print(delito)
    mask_ = ((df_gen_resumen_conflictividad['CONFLICTIVIDAD']==delito))
    print(len(mask_))
    
    df_gen_resumen_conflictividad_delito = pd.DataFrame() 
    df_gen_resumen_conflictividad_delito = df_gen_resumen_conflictividad[mask_]
    print('tamaño del df: ',df_gen_resumen_conflictividad_delito.shape)


NameError: name 'lista_delitos' is not defined